In [ ]:
import numpy as np
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sys
sys.path.append('../zdrojaky')
from tsplot import *

# AR(2) model ve `statsmodels`

Autoregresní model druhého řádu má tvar

$$
Y_t = \phi_1 Y_{t-1} + \phi_2 Y_{t-2} + \varepsilon_t, \qquad \varepsilon_t \sim \mathcal{N}(0, \sigma^2)
$$

Označme
$$
\mathbf{X}_t = 
\begin{bmatrix}
Y_t \\ Y_{t-1}
\end{bmatrix}
,\qquad
\mathbf{w}_t =
\begin{bmatrix}
\varepsilon_t \\ 0
\end{bmatrix},
$$

potom

$$
\begin{align}
Y_t &= [1, 0]\, \mathbf{X}_t \\
\mathbf{X}_t &=
\begin{bmatrix}
\phi_1 & \phi_2 \\
1 & 0
\end{bmatrix}
\mathbf{X}_{t-1} + \mathbf{w}_t.
\end{align}
$$

Parametry jsou tedy $\sigma^2, \phi_1, \phi_2$

In [ ]:
class AR2Model(sm.tsa.statespace.MLEModel):
    def __init__(self, endog):
        # Model order
        k_states = ***
        k_posdef = ***

        # Initialize the statespace
        super(AR2Model, self).__init__(
            endog, k_states=k_states, k_posdef=k_posdef,
            initialization='approximate_diffuse',
            loglikelihood_burn=k_states
        )

        # Initialize the matrices
        self.ssm['design', ***] = ***
        self.ssm['transition', ***] = ***
        self.ssm['selection', ***] = ***
        

    @property
    def param_names(self):
        return ['sigma2.process', 'phi.1', 'phi.2']

    @property
    def start_params(self):
        return [np.std(self.endog), 0, 0]

    def transform_params(self, params):
        sigma2 = params[0]**2
        return np.r_[sigma2, params[1], params[2]]

    def untransform_params(self, params):
        sigma2 = params[0]**0.5
        return np.r_[sigma2, params[1], params[2]]

    def update(self, params, *args, **kwargs):
        params = super(AR2Model, self).update(params, *args, **kwargs)
        
        # State noise covariance
        self.ssm['state_cov', ***] = params[***]

        # Transition matrix
        self.ssm['transition', ***] = params[***]   

## Simulovaná testovací data

Připravíme si testovací data AR(2) procesu s šumem $\varepsilon_t \sim \mathcal{N}(0, 0.5)$ a koeficienty $\phi_1 = -0.8, \phi_2 = 0.2$.

In [ ]:
#%% Simulace
np.random.seed(12)
ndat = 200
noise_var = 0.5
noise = np.random.normal(***)   # Zde bude ulozen sum, at jej nemusime generovat v cyklu nize

Y = np.zeros(ndat)
Y[0:2] = np.array([.8, .5])     # Pocatecni hodnoty
phi = np.array([-.8, .2])
for t in range(2, ndat):
    Y[t] = *** + noise[t]

tsplot(Y)

Zkusíme je prohnat modelem a odhadnout "neznámé parametry" z přímo z dat.

In [ ]:
model = AR2Model(Y)
res = model.fit()
print(res.summary())

Stejně jako v předchozím příkladu zkusíme rovněž udělat predikce na dalších `nsteps = 20` kroků dopředu.

In [ ]:
nsteps = 20
predict = res.get_prediction()
forecast = res.get_forecast(nsteps)

plt.figure(figsize=(15,4))
plt.plot(Y, label='Y')
plt.plot(predict.predicted_mean, label='Predicted')
predict_ci = predict.conf_int(alpha=0.05)
predict_index = np.arange(len(predict_ci))
plt.fill_between(predict_index[2:], predict_ci.iloc[2:, 0], predict_ci.iloc[2:, 1], color='grey', alpha=0.1)
plt.plot(np.arange(ndat, ndat+nsteps), forecast.predicted_mean, label='Forecast')
forecast_ci = forecast.conf_int()
forecast_index = np.arange(len(predict_ci), len(predict_ci) + len(forecast_ci))
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='green', alpha=0.1)
plt.legend()
plt.show()